# The Peer-graded Assignment:
# Segmenting and Clustering Neighborhoods in Toronto

### Instructions
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.
***



## Step One: Scrape Data for Toronto

There is not existing data avaible for the Neighborhoods in Toronoto, therefore the first step of the assignment is to extract data from website. 

### Tips for  Webscraping Updated Table in Week3 Peer Graded Assignment

 **After retreiving the URL and creating a Beautiful soup object** 

 **Firstly create a list**  

 **Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

**As postal code contains upto 3 characters extract that using tablerow.p.text**

 **Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

 **Append to the list**  

 **Create a dataframe with list**


### Workspace Enviroment Setup
***** Download all the frame for the project *****

In [117]:
# The code was removed by Watson Studio for sharing.

In [54]:
#********* Framework and Liberays *****************
#******** Data Handling **************
from bs4 import BeautifulSoup as bfs # Beatifulsoup for Scraping
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#*** Data Analysis tool ********
# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests # library to handle requests
import time #for delays

#****** Geomatric server
!pip install geocoder
!pip install geopy
!pip install geopandas
!pip install pgeocode #the google map decoder
# Installed liberay

import geocoder # import geocoder
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pgeocode as pgo




#********* The Data Source URL ***********
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

print("Ready !!!")

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

### Scrape from target 

In [2]:
# **** Save the HTML Source code ******
html_data = requests.get(URL).text
# print(html_data)

#**** Create the Beatifulsoup objecy *****
soup = bfs(html_data,"html5lib")
# soup.tbody

# #read html with panda directly
# dfr = pd.read_html(html_data,match='.+')
# print(dfr)

### Extract Data and Preprocess

In [3]:
# Extract Data from HTML and creat a list
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

# Convert list to dataframe for further process
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df = df.sort_values(by=['PostalCode'])
df.reset_index(inplace=True)
df = df.drop(columns='index',axis=1)


***

## Step Two:  Find Coordination
Now that after built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 




In [12]:

postal_code = df['PostalCode'] #Extract Post from dataframe
# geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
locator = pgo.Nominatim("CA")
ldf = pd.DataFrame(columns = ['Latitude','Longitude']) #Create a new Dataframe for Coordination
# print(postal_code)

#******* find coordinate for each post code *********
for code in postal_code:
    
# initialize your variable to None
    location = None

# loop until you get the coordinates
    print(code)
    while(location is None or location is 'NaN'):
       
        location = locator.query_postal_code("{} Toronto Ontario".format(code))
        time.sleep(1)
    #Record the respect co-ordinate of each post code to the new table
    ldf = ldf.append({ "Latitude": location.latitude,"Longitude": location.longitude },ignore_index = True)
    
ldf 




          

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


,Latitude,Longitude
0,43.8113,-79.1930
1,43.7878,-79.1564
2,43.7678,-79.1866
3,43.7712,-79.2144
4,43.7686,-79.2389
5,43.7464,-79.2323
6,43.7298,-79.2639
7,43.7122,-79.2843
8,43.7247,-79.2312
9,43.6952,-79.2646


In [20]:
# Tidy up the table and merge the table into one for the Toronoto Data.
fdf = pd.merge(df, ldf,right_index=True, left_index=True)
fdf.loc[86,['Latitude','Longitude']] = [43.6369,-79.6158] #replace the coordination missing
fdf


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
5,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.7122,-79.2843
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.7247,-79.2312
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6952,-79.2646


***

## Step Three: Clustering

### Create a map of Toronto

In [6]:
# find the coordination of the Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto")
location_t = geolocator.geocode(address)
latitude_t = location_t.latitude
longitude_t = location_t.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_t, longitude_t))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [38]:
#Creat a map of Toronto with the coordination from above

map_toronto = folium.Map(location_t=[latitude_t, longitude_t], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(fdf['Latitude'], fdf['Longitude'], fdf['Borough'], fdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Explore Downtown Toronto

There are many Borough and neighborhood in Toronot. But Downtown Toronto should be the most popular destination in Toronto.


In [43]:
# Locate all the neighborhood in Downtown
dt_df = fdf[fdf['Borough'] == 'Downtown Toronto'].reset_index(drop= True) # extra all the neigborhood within Downtown Toronto
dt_df
#find the coordination of Downtown
address = 'Downtown, Toronto'

geolocator = Nominatim(user_agent="DT_Toronto")
location_dt = geolocator.geocode(address)
latitude_dt = location_dt.latitude
longitude_dt = location_dt.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_dt, longitude_dt))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081162653639.


### 1.  list all the neighborhood in the Downtown area

In [45]:
#Draw map for Downtown Toronto
map_dt = folium.Map(location_dt=[latitude_dt, longitude_dt], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dt_df['Latitude'], dt_df['Longitude'], dt_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt)  
    
map_dt

### 2 Explore Neighborhoods in Toronto


In [51]:
#Creat Class for exploring nearby Venues

class VenExp:
    def __init__(self, names, latitudes, longitudes,C_ID,C_SEC): 
        self.names = names
        self.latitudes = latitudes
        self.longitudes = longitudes
        self.C_ID = C_ID
        self.C_SEC = C_SEC
        
        #the private variable of the class
        VERSION = '20180605' # Foursquare API version
        LIMIT = 100 # A default Foursquare API limit value
        radius = 500 #venue within the radius of 500m
        venues_list=[]
    
    def getNearbyVenues(self):
    
        # The loggin infor for the API
#         #the private variable of the class
#         VERSION = '20180605' # Foursquare API version
#         LIMIT = 100 # A default Foursquare API limit value
#         radius = 500 #venue within the radius of 500m
#         venues_list=[]
        
    
    
        for name, lat, lng in zip(self.names, self.latitudes, self.longitudes):
            print(name)
            
            # create the API request URL
        
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                self.C_ID, 
                self.C_SEC, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    
        return(nearby_venues)
    

#### Explore the neighborhoods and Created New Dataframe in Downtown

In [57]:
#predefined the variables
name_d = dt_df['Neighborhood']
latitudes_D = dt_df['Latitude']
longitudes = dt_df['Longitude']
C_ID = CLIENT_ID
C_SEC = CLIENT_SECRET

#create a class object downtown_venue
downtown_venue = VenExp(name_d,
                        latitudes_D,
                        longitudes,
                        C_ID,
                        C_SEC
                       )

In [58]:
venue_in_dt = downtown_venue.getNearbyVenues() #the venue in downtown data frame

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
First Canadian Place, Underground city
Christie


In [63]:
print('The size of the table',venue_in_dt.shape)
venue_in_dt.head()


The size of the table (1001, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.6827,-79.3730,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.6827,-79.3730,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.6827,-79.3730,Scoops Convenience Boutique,43.686148,-79.375828,Candy Store
3,Rosedale,43.6827,-79.3730,Rosedale Park,43.682328,-79.378934,Playground
4,"St. James Town, Cabbagetown",43.6684,-79.3689,Cabbagetown Brew,43.666923,-79.369289,Café


#### Number of Venues are in each Neighborhood within the Downtown

In [64]:
venue_in_dt.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,75,75,75,75,75,75
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",58,58,58,58,58,58
Central Bay Street,57,57,57,57,57,57
Christie,10,10,10,10,10,10
Church and Wellesley,68,68,68,68,68,68
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",5,5,5,5,5,5


#### The number of types of venues in Downtown Toronto 

In [65]:
print('There are {} uniques categories of Venues.'.format(len(venue_in_dt['Venue Category'].unique())))

There are 158 uniques categories of Venues.


In [69]:
# one hot encoding, dt_ohe means downtown_one_hot_encoding
dt_ohe = pd.get_dummies(venue_in_dt[['Venue Category']], prefix="", prefix_sep="")

# print(dt_ohe)
# add neighborhood column back to dataframe
dt_ohe['Neighborhood'] = venue_in_dt['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_ohe.columns[-1]] + list(dt_ohe.columns[:-1])
dt_ohe = dt_ohe[fixed_columns]

print(dt_ohe.shape)
dt_ohe.head()

(1001, 158)


,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [73]:
dt_grouped = dt_ohe.groupby('Neighborhood').mean().reset_index()
print('size of data',dt_grouped.shape)
dt_grouped #downtown venues grouped  

size of data (17, 158)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.000000,0.0,0.013333,0.040000,0.026667,0.000000,0.013333,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013333,0.013333,0.000000,0.040000,0.000000,0.000000,0.013333,0.000000,0.013333,0.040000,0.053333,0.000000,0.000000,0.00,0.00,0.013333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026667,0.000000,0.00,0.000000,0.00,0.00,0.013333,0.000000,0.00,0.000000,0.013333,0.000000,0.00,0.000000,0.013333,0.000000,0.013333,0.000000,0.0,0.000000,0.000000,0.00,0.053333,0.013333,0.00,0.000000,0.013333,0.026667,0.040000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333,0.013333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.013333,0.00,0.000000,0.013333,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.013333,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.00,0.013333,0.000000,0.00,0.013333,0.000000,0.013333,0.013333,0.066667,0.026667,0.000000,0.00,0.000000,0.013333,0.013333,0.000000,0.026667,0.013333,0.000000,0.013333,0.013333,0.000000,0.00,0.013333,0.000000,0.000000,0.00,0.00,0.026667,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.0,0.000000,0.034483,0.034483,0.034483,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.017241,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086207,0.000000,0.017241,0.00,0.00,0.000000,0.

##### Top 5 most common venues in each neighborhood

In [75]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0  Sandwich Place  0.07
1     Coffee Shop  0.05
2           Hotel  0.05
3    Cocktail Bar  0.04
4        Beer Bar  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0           Coffee Shop  0.09
1    Italian Restaurant  0.07
2  Gym / Fitness Center  0.03
3                   Bar  0.03
4             Speakeasy  0.03


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.21
1       Sandwich Place  0.09
2          Pizza Place  0.05
3     Sushi Restaurant  0.05
4  Japanese Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store   0.3
1           Café   0.3
2           Bank   0.1
3           Park   0.1
4     Baby Store   0.1


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.07
1     Sushi Restaurant  0.07
2              Gay Bar  0.04
3           Restaurant  0.04
4  

#### Put common venues in Dataframe
A function to sort the venues in descending order.

In [77]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


#### Create the new dataframe and display the top 10 venues for each neighborhood

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Sandwich Place,Hotel,Coffee Shop,Beer Bar,Bakery,Japanese Restaurant,Cocktail Bar,Café,Bank,Sporting Goods Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Wings Joint,Speakeasy,Bakery,Bank,Bar,French Restaurant,Park,Pizza Place
2,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Café,Gym / Fitness Center,Breakfast Spot
3,Christie,Grocery Store,Café,Baby Store,Coffee Shop,Bank,Park,Gastropub,Gas Station,Dog Run,Distribution Center
4,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Coffee Shop,Men's Store,Mediterranean Restaurant,Indian Restaurant,Burrito Place


## 3. Cluster Neighborhoods
#### Run k-means to cluster the downtown neighborhood into 6 clusters.

In [93]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans_dt = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_dt.labels_[0:10] 

array([3, 3, 0, 4, 3, 3, 3, 3, 1, 3], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [97]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_dt.labels_)

downtown_merged = dt_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels 1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.6827,-79.3730,2,1,Park,Grocery Store,Playground,Candy Store,Deli / Bodega,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.6684,-79.3689,3,5,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Chinese Restaurant,Pizza Place,Café,Sandwich Place,Japanese Restaurant,Jewelry Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830,3,5,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Coffee Shop,Men's Store,Mediterranean Restaurant,Indian Restaurant,Burrito Place
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,3,Coffee Shop,Thai Restaurant,Event Space,Italian Restaurant,Beer Store,Playground,Restaurant,Pub,Bakery,Dance Studio
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,3,0,Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Bank,Middle Eastern Restaurant,Japanese Restaurant,Pizza Place,Cosmetics Shop


 ### Visualize the resulting clusters

In [98]:
# create map
map_clusters = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster1 ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5.Examine Clusters
#### examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [111]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels 1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,"Garden District, Ryerson",3,0,Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Bank,Middle Eastern Restaurant,Japanese Restaurant,Pizza Place,Cosmetics Shop
8,Downtown Toronto,"Richmond, Adelaide, King",3,0,Coffee Shop,Café,Sandwich Place,Restaurant,Sushi Restaurant,Asian Restaurant,Hotel,Gym,Deli / Bodega,Bank
10,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",3,0,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Deli / Bodega,Restaurant,Salad Place,Japanese Restaurant,Pharmacy
11,Downtown Toronto,"Commerce Court, Victoria Hotel",3,0,Coffee Shop,Café,Sandwich Place,Hotel,Restaurant,Bank,Asian Restaurant,Japanese Restaurant,Gym,Deli / Bodega
12,Downtown Toronto,"University of Toronto, Harbord",0,0,Coffee Shop,Café,Bakery,Sandwich Place,Japanese Restaurant,Bar,Pub,Bank,Restaurant,Comfort Food Restaurant
15,Downtown Toronto,"First Canadian Place, Underground city",3,0,Coffee Shop,Café,Sandwich Place,Hotel,Restaurant,Bank,Asian Restaurant,Japanese Restaurant,Gym,Deli / Bodega


#### Cluster 2

In [112]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels 1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Rosedale,2,1,Park,Grocery Store,Playground,Candy Store,Deli / Bodega,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 3

In [113]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels 1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",1,2,Park,Café,Harbor / Marina,Music Venue,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 4

In [107]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels 1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,"Regent Park, Harbourfront",0,3,Coffee Shop,Thai Restaurant,Event Space,Italian Restaurant,Beer Store,Playground,Restaurant,Pub,Bakery,Dance Studio
7,Downtown Toronto,Central Bay Street,0,3,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Café,Gym / Fitness Center,Breakfast Spot


#### Cluster 5

In [106]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1,2] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels 1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Downtown Toronto,Christie,4,4,Grocery Store,Café,Baby Store,Coffee Shop,Bank,Park,Gastropub,Gas Station,Dog Run,Distribution Center


# END!
***